# ch 13. Using segmentation to find suspected nodules

---

이번 챕터에서는 프로젝트에 세그멘테이션 모델을 추가한다.

기존 모델은 사람이 직접 분류하고 답을 분류해놓은 데이터를 이용했었다. 세그멘테이션 모델이 있다면 이 과정까지 모델에 맡길 수 있다.


### 세그멘테이션의 유형

1. **시맨틱 세그멘테이션**(**Semantic Segmentation**)
   - 이번 장에서 우리가 사용할 유형이다. 레이블을 사용해서 이미지의 개별 픽셀을 분류하는 방식으로 동작한다.
2. **인스턴스 세그멘테이션**(**Instance Segmentation**)
   - 이 방법은 사람이 여러 명 있는 사진 등을 구분할 때 사용한다. 사람1, 사람2와 같은 방식으로 동일한 객체가 많을 때 이를 구분하는게 필요할 때 사용한다.
3. **객체 탐지**(**Object Detection**)
   - 대표적인 모델로 YOLO 모델이 있다. 주어진 이미지에서 관심있는 대상에 박스를 쳐서 나타내준다.


기존에 우리가 구현했었던 분류 모델은 주어진 이미지를 하나의 포인트로 집결시키고 이를 토대로 분류한다. 즉 귀, 눈, 코와 같은 고차원 개념의 특성을 찾아내서 무엇인지 분류해낸다. 이 때 다운샘플링을 거치면서 컨볼루션의 수용필드가 증가하면서 더 넓은 영역에서도 고차원의 특성을 찾아낼 수 있게 된다.

만약 출력과 입력이 같은 크기여야 한다면 이와는 다른 모델을 사용해야 한다. 단순하게 생각하면 다운샘플링을 거치치 않는 연속적 컨볼루션 구성을 생각해볼 수 있다. 하지만 이렇게 할 경우 수용필드에 한계가 생긴다. 수용필드가 입력 픽셀과 동일한 출력 픽셀을 유지하면서 어떻게 출력 픽셀의 수용필드를 개선할 수 있을까?

**업샘플링**(**Upsampling**)을 사용하면된다. 주어진 이미지의 해상도보다 높은 해상도의 이미지를 만들어내는 것이다.

## 유넷(U-Net) 아키텍처

유넷의 아키텍처는 아래 그림과 같다. 데이터는 먼저 좌상단에서 하단 중앙부로 이동하면서, 여러 컨볼루션 층과 다운 스케일을 거친다. 그 후 다시 오른쪽 위로 올라가면서 업스케일링과 컨볼루션 층을 거치고 전체 해상도로 돌아온다. 하지만 이렇게 되면 수렴 문제가 발생한다. 즉, 다운샘플링을 거치는 동안 공간 정보를 손실하기 때문에 사물의 경계부분의 정확한 위치를 인코딩해서 재구성하기가 어려워진다.

이를 해결하기 위해 유넷은 스킵 커넥션을 사용한다. 이로 인해 뒷부분에서는 아래 부분에서 나온 업샘플링 결과와 스킵 커넥션을 거치는 출력을 동시에 받게 된다. 이 부분이 유넷의 핵심이다. 유넷은 두 데이터 중에서 가장 나은 데이터(정보)를 사용해서 연산한다.


![image](https://user-images.githubusercontent.com/76675506/191262454-8b37e6ea-a570-4794-b803-f7c968cbfc67.png)

### 오픈 소스를 활용하기

---

유넷은 jvanvugt/pytorch-unet에 공개된 오픈 소스를 활용해서 구현한다. 여기서 필요한 부분만 수정해서 사용한다.

순정 모델에서 요소를 하나씩 제거하면서 결과를 실험해보는 걸 **제거 연구**(**Ablation study**)라고도 한다. (https://cumulu-s.tistory.com/8)

코드 출처: https://github.com/jvanvugt/pytorch-unet/blob/master/unet.py

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F


class UNet(nn.Module):
    def __init__(
        self,
        in_channels=1,
        n_classes=2,
        depth=5,
        wf=6,
        padding=False,
        batch_norm=False,
        up_mode='upconv',
    ):
        """
        Implementation of
        U-Net: Convolutional Networks for Biomedical Image Segmentation
        (Ronneberger et al., 2015)
        https://arxiv.org/abs/1505.04597
        Using the default arguments will yield the exact version used
        in the original paper
        Args:
            in_channels (int): number of input channels
            n_classes (int): number of output channels
            depth (int): depth of the network
            wf (int): number of filters in the first layer is 2**wf
            padding (bool): if True, apply padding such that the input shape
                            is the same as the output.
                            This may introduce artifacts
            batch_norm (bool): Use BatchNorm after layers with an
                               activation function
            up_mode (str): one of 'upconv' or 'upsample'.
                           'upconv' will use transposed convolutions for
                           learned upsampling.
                           'upsample' will use bilinear upsampling.
        """
        super(UNet, self).__init__()
        assert up_mode in ('upconv', 'upsample')
        self.padding = padding
        self.depth = depth
        prev_channels = in_channels
        self.down_path = nn.ModuleList()
        for i in range(depth):
            self.down_path.append(
                UNetConvBlock(prev_channels, 2 ** (wf + i), padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.up_path = nn.ModuleList()
        for i in reversed(range(depth - 1)):
            self.up_path.append(
                UNetUpBlock(prev_channels, 2 ** (wf + i), up_mode, padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.last = nn.Conv2d(prev_channels, n_classes, kernel_size=1)

    def forward(self, x):
        blocks = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            if i != len(self.down_path) - 1:
                blocks.append(x)
                x = F.max_pool2d(x, 2)

        for i, up in enumerate(self.up_path):
            x = up(x, blocks[-i - 1])

        return self.last(x)


class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, padding, batch_norm):
        super(UNetConvBlock, self).__init__()
        block = []

        block.append(nn.Conv2d(in_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        block.append(nn.Conv2d(out_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        self.block = nn.Sequential(*block)

    def forward(self, x):
        out = self.block(x)
        return out


class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, up_mode, padding, batch_norm):
        super(UNetUpBlock, self).__init__()
        if up_mode == 'upconv':
            self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2, stride=2)
        elif up_mode == 'upsample':
            self.up = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_size, out_size, kernel_size=1),
            )

        self.conv_block = UNetConvBlock(in_size, out_size, padding, batch_norm)

    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[
            :, :, diff_y : (diff_y + target_size[0]), diff_x : (diff_x + target_size[1])
        ]

    def forward(self, x, bridge):
        up = self.up(x)
        crop1 = self.center_crop(bridge, up.shape[2:])
        out = torch.cat([up, crop1], 1)
        out = self.conv_block(out)

        return out

아래 코드는 우리가 프로젝트에서 사용할 코드다. 먼저 입력 데이터를 배치 정규화한다. 이를 통해 데이터셋을 직접 정규화할 필요가 없어진다. 또한 개별 배치에 대해 통계량을 얻을 수 있다. 즉, 신경망에 들어간 크롭된 CT 데이터에서 아무것도 확인할 수 없을 때 비율을 조정할 수 있게 된다.


그 후 출력을 `nn.Sigmoid`를 통해 범위를 [0,1]로 제한한다. 다음으로는 모델 자체의 전체 깊이와 필터수를 줄인다. 현재 모델은 우리가 사용할 데이터에 비해 capacity가 크기 때문이다.

In [ ]:
# model.py:17, class UNetWrapper

class UNetWrapper(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()

        self.input_batchnorm = nn.BatchNorm2d(kwargs['in_channels'])
        self.unet = UNet(**kwargs)
        self.final = nn.Sigmoid()

        self._init_weights()

    def forward(self, input_batch): # nn.sequential을 사용할 수 있지만 유지보수를 위해 명시적으로 작성했다.
        bn_output = self.input_batchnorm(input_batch)
        un_output = self.unet(bn_output)
        fn_output = self.final(un_output)
        return fn_output

### 세그멘테이션을 위한 데이터셋 업데이트

---

위 코드에서 `nn.BatchNorm2d`을 사용한다는 점에 주목하자. 유넷은 기본적으로 2차원 세그멘테이션 모델이다. 이를 사용하려면 우리 데이터(3차원)를 단면으로 나누고 한 번에 한 단면에 대해서 인접한 단면을 제공한다. 축 방향으로 존재하던 공간 정보(인접 관계)를 잃게 되지만, 모델이 이를 다시 학습하는건 어렵지 않다.


또한, 유넷은 입출력 크기가 매우 제한적이다. CT 단면(512x512)보다 입력(572x572)은 크지만 출력(388x388)은 굉장히 작다. 이를 위해 유넷에 `padding` 파라미터를 `True`로 설정한다. 이로 인해 정확도가 떨어질 순 있지만 감수해야 하는 부분이다.

또 해결해야 할 부분이 있다. 사람이 레이블한 훈련 데이터와 모델의 출력값이 맞지 않는다는 사실이다. `annotation.csv`에 좌표 정보가 있지만 복셀 단위의 마스크 정보가 필요하다.


#### 바운딩 박스

결절 전체를 둘러싸는 **바운딩 박스**(**bounding box**)에 넣는 작업부터 시작한다. 이 작업은 실제 결절에 대해서만 수행한다. 작업의 알고리즘은 아래 그림과 같다. O로 표시된 부분을 중심으로 탐색한다. 코드는 아래와 같다.


![image](https://user-images.githubusercontent.com/76675506/191271071-8563824c-a5f7-4208-b784-bfdb9405f935.png)

In [ ]:
#  dsets.py:131, Ct.buildAnnotationMask

def buildAnnotationMask(self, positiveInfo_list, threshold_hu = -700): # 바운딩 박스 마스크가 경계값인 -700HU보다 밀도가 높은 조직에 겹치는 경우에 다듬어준다. 이를 통해 윤곽을 깔끔하게 만들어준다.
    boundingBox_a = np.zeros_like(self.hu_a, dtype=np.bool)

    for candidateInfo_tup in positiveInfo_list:
                center_irc = xyz2irc(
                    candidateInfo_tup.center_xyz,
                    self.origin_xyz,
                    self.vxSize_xyz,
                    self.direction_a,
                )
                ci = int(center_irc.index) # 중심점 복셀 인덱스
                cr = int(center_irc.row)
                cc = int(center_irc.col)

                index_radius = 2
                try:
                    while self.hu_a[ci + index_radius, cr, cc] > threshold_hu and \ # 위 그림에서 설명한 과정
                            self.hu_a[ci - index_radius, cr, cc] > threshold_hu:
                        index_radius += 1
                except IndexError:
                    index_radius -= 1

                boundingBox_a[
                             ci - index_radius: ci + index_radius + 1,
                             cr - row_radius: cr + row_radius + 1,
                             cc - col_radius: cc + col_radius + 1] = True

            mask_a = boundingBox_a & (self.hu_a > threshold_hu)

            return mask_a

![image](https://user-images.githubusercontent.com/76675506/191501634-6ab250bb-460c-4589-ab82-379cb7310eb8.png)


### CT 초기화 때 마스크 생성 호출하기

이제 결절 정보를 가진 튜플 리스트를 결절인지 아닌지 분류한 값으로 나타낼 수 있다.

실제 결절만 포함하는 값을 필터링한 후 애노테이션 마스크를 만들자.

그 후 최소 하나라도 결절이 들어있는 배열 인덱스로 만들어 이를 검증용 데이터를 만들 때 활용한다.

또한 앞선 장에서 모든 CT 데이터를 읽어서 파싱할 필요가 없게 결절 후보 데이터를 이용해 CT를 캐싱했다. 여기서도 동일하게 만든다.

In [ ]:
# dsets.py:99, Ct.__init__

class Ct:
    def __init__(self, series_uid):

        candidateInfo_list = getCandidateInfoDict()[self.series_uid]

        self.positiveInfo_list = [
            candidate_tup
            for candidate_tup in candidateInfo_list
            if candidate_tup.isNodule_bool
        ]
        self.positive_mask = self.buildAnnotationMask(self.positiveInfo_list)
        self.positive_indexes = (self.positive_mask.sum(axis=(1,2)) # 단면마다 몇개의 복셀 마스크가 있는지 넘겨줌
                                 .nonzero()[0].tolist())

### 애노테이션 데이터 정제

애노테이션 데이터를 더 나은 방법으로 선별할 수 있다.

`candidate.csv`에 나열된 몇몇 후보데이터는 파일에 중복해서 나타난다. 그런데 애노테이션은 완전히 동일한 데이터가 아니다.

약간의 트릭을 써서 `annotation.csv` 파일을 클린업해보자.

아래 코드의 `getCandidateInfoList`를 이용해서 구현할 수 있다.

In [ ]:
# dsets.py:43, def getCandidateInfoList
def getCandidateInfoList(requireOnDisk_bool=True):
    candidateInfo_list = []
    with open('data/part2/luna/annotations_with_malignancy.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])
            isMal_bool = {'False': False, 'True': True}[row[5]]

            candidateInfo_list.append(
                CandidateInfoTuple(
                    True,
                    True,
                    isMal_bool,
                    annotationDiameter_mm,
                    series_uid,
                    annotationCenter_xyz,
                )
            )
